In [2]:
import scipp as sc
import numpy as np
from scipp import curve_fit
from functools import partial
%matplotlib widget

def fit_function(energy,*,amp,width):
    y=amp*sc.exp(-0.5*(energy/width)**2)
    return y

x=sc.linspace('energy',-1,1,101,unit='meV')
y=fit_function(x,amp=1*sc.Unit('meV'),width=0.1*sc.Unit('meV'))
noise=sc.array(dims=['energy'],values=0.1*np.random.randn(101),unit='meV')
da=sc.DataArray(y+noise,coords={'energy':x})

popt, _ = curve_fit(['energy'], fit_function, da, p0={'amp': 1.0*sc.Unit('meV'), 'width': 0.1*sc.Unit('meV')})
(popt)


DataGroup(sizes={}, keys=[
    width: DataArray({}),
    amp: DataArray({}),
])

In [3]:
fixed_amp=partial(fit_function,amp=1*sc.Unit('meV'))
popt_fixed_amp, _ = curve_fit(['energy'],fixed_amp, da, p0={'width': 0.1*sc.Unit('meV')})
(popt_fixed_amp)

DataGroup(sizes={}, keys=[
    width: DataArray({}),
    amp: DataArray({}),
])

In [5]:
import scipp as sc
import numpy as np
from scipp import curve_fit
from functools import partial
%matplotlib widget

def fit_function(energy, *, amp, width):
    y = amp * sc.exp(-0.5 * (energy / width) ** 2)
    return y

# Data preparation
x = sc.linspace('energy', -1, 1, 101, unit='meV')
y = fit_function(x, amp=1 * sc.Unit('meV'), width=0.1 * sc.Unit('meV'))
noise = sc.array(dims=['energy'], values=0.1 * np.random.randn(101), unit='meV')
da = sc.DataArray(y + noise, coords={'energy': x})

# First curve fit with both parameters free
popt, _ = curve_fit(['energy'], fit_function, da, p0={'amp': 1.0 * sc.Unit('meV'), 'width': 0.1 * sc.Unit('meV')})
print("Fit results (both parameters free):", popt)

# Fix the amp parameter using partial
fixed_amp = partial(fit_function, amp=1 * sc.Unit('meV'))

# Second curve fit with amp fixed
popt_fixed_amp, _ = curve_fit(['energy'], fixed_amp, da, p0={'width': 0.1 * sc.Unit('meV')})
print("Fit results (amp fixed):", popt_fixed_amp)
popt_fixed_amp

Fit results (both parameters free): DataGroup(sizes={}, keys=['width', 'amp'])
Fit results (amp fixed): DataGroup(sizes={}, keys=['width', 'amp'])


DataGroup(sizes={}, keys=[
    width: DataArray({}),
    amp: DataArray({}),
])